# Part 2: Data cleaning

This notebook shows the approaches for treating missing data for furture analysis and modeling
1. Combined all scraped files into one dataframe and check missing values
2. Drop and compute missing values for each variable
    * **Drop** variables that are not meaningful,duplicates
    * **Correct** outliers due to data error
    * **Impute** using only the target variable non-missing values 
    * **Impute** the target variable missing values using other related variables



In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import glob
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 30)

In [4]:
#Find all sold files
soldfiles=glob.glob("*sold.csv")
soldfiles

['zillow-78613-sold.csv',
 'zillow-78641-sold.csv',
 'zillow-78660-sold.csv',
 'zillow-78664-sold.csv']

In [10]:
#Combine all sold files to one dataframe
df = pd.concat([pd.read_csv(i) for i in soldfiles ],sort=False,ignore_index=True)
print(df.shape)
df.head(3)

(5617, 22)


,Unnamed: 0,beds,baths,sqft,solddate,soldprice,estimateprice,housetype,YearBuilt,Heating,Cooling,parking,lotsize,name,numberOfRooms,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,houseurl
0,0.0,3.0,2.0,1478.0,4/22/2019,224778.0,243589.0,Single Family,2000,Other,Central,2,5662.0,"702 Le Ann Ln, Cedar Park, TX 78613",3.0,702 Le Ann Ln,Cedar Park,TX,78613,30.518779,-97.817411,https://www.zillow.com/homedetails/702-Le-Ann-...
1,1.0,4.0,2.5,2665.0,4/12/2019,250000.0,257730.0,Single Family,2000,Forced air,Central,1,8276.0,"2605 Byfield Dr, Cedar Park, TX 78613",4.0,2605 Byfield Dr,Cedar Park,TX,78613,30.540536,-97.845924,https://www.zillow.com/homedetails/2605-Byfiel...
2,2.0,3.0,3.0,2255.0,4/5/2019,269000.0,270245.0,Single Family,2000,"Forced air, Other",Central,2,7666.0,"1622 Abbey Ln, Cedar Park, TX 78613",3.0,1622 Abbey Ln,Cedar Park,TX,78613,30.522559,-97.854897,https://www.zillow.com/homedetails/1622-Abbey-...


In [11]:
(df.isna()).sum()

Unnamed: 0         4392
beds                577
baths                93
sqft                  9
solddate              0
soldprice           167
estimateprice        92
housetype             0
YearBuilt             0
Heating               0
Cooling               0
parking               0
lotsize             606
name                  0
numberOfRooms       324
streetAddress         0
addressLocality       0
addressRegion         0
postalCode            0
latitude              0
longitude             0
houseurl              0
dtype: int64

# soldprice
We first drop columns that are not needed. We also have to drop observations that don't have sold price since we will use it to analyze with other variables. The missing values of sold price is also low, so it is safe to drop.

In [12]:
df=df.drop(['Unnamed: 0',"estimateprice"],axis=1)
df=df[~df.soldprice.isna()]
df.reset_index(inplace = True, drop = True)

## Sqft
There are 9 missing data from **sqft**. The dataframe below shows that we also have missing data from other variable when sqft is missing. We will simply drop the missing value since we need to use this to impute other missing variables

In [14]:
df[df.sqft.isna()]

,beds,baths,sqft,solddate,soldprice,housetype,YearBuilt,Heating,Cooling,parking,lotsize,name,numberOfRooms,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,houseurl
767,3.0,NaN,NaN,7/21/2016,247000.0,Single Family,2009,Forced air,No Data,2,6534.00,"1911 Sand Creek Rd, Cedar Park, TX 78613",3.0,1911 Sand Creek Rd,Cedar Park,TX,78613,30.520134,-97.795066,https://www.zillow.com/homedetails/1911-Sand-C...
1153,NaN,NaN,NaN,6/19/2018,539900.0,Single Family,2017,Other,Central,No Data,NaN,"309 Parke Wind Way, Cedar Park, TX 78613",0.0,309 Parke Wind Way,Cedar Park,TX,78613,30.504828,-97.846678,https://www.zillow.com/homedetails/309-Parke-W...
1964,4.0,3.0,NaN,5/24/2017,245652.0,Single Family,2009,Electric,No Data,2,5000.00,"796 Kingfisher Ln, Leander, TX 78641",4.0,796 Kingfisher Ln,Leander,TX,78641,30.631102,-97.873208,https://www.zillow.com/homedetails/796-Kingfis...
2862,NaN,NaN,NaN,8/13/2018,310000.0,Single Family,2017,Other,Central,No Data,NaN,"200 Allegrini St, Leander, TX 78641",NaN,200 Allegrini St,Leander,TX,78641,30.567123,-97.885847,https://www.zillow.com/homedetails/200-Allegri...
2873,NaN,NaN,NaN,6/14/2018,436947.0,Single Family,2017,Other,Central,No Data,0.27,"2429 Adriel Cv, Leander, TX 78641",NaN,2429 Adriel Cv,Leander,TX,78641,30.599296,-97.818128,https://www.zillow.com/homedetails/2429-Adriel...
2874,NaN,NaN,NaN,6/8/2018,234539.0,Single Family,2017,Other,Central,No Data,NaN,"249 Trellis Blvd, Leander, TX 78641",0.0,249 Trellis Blvd,Leander,TX,78641,30.634505,-97.863316,https://www.zillow.com/homedetails/249-Trellis...
2883,NaN,NaN,NaN,5/21/2018,61130.0,Single Family,2017,Other,Central,No Data,NaN,"144 Syrah Ct, Leander, TX 78641",0.0,144 Syrah Ct,Leander,TX,78641,30.634642,-97.863902,https://www.zillow.com/homedetails/144-Syrah-C...
2904,NaN,NaN,NaN,4/1/2018,69813.0,Single Family,2017,Other,Central,No Data,NaN,"141 Hopen Cv, Leander, TX 78641",0.0,141 Hopen Cv,Leander,TX,78641,30.622704,-97.868795,https://www.zillow.com/homedetails/141-Hopen-C...
3018,NaN,6.0,NaN,9/28/2017,837229.0,Single Family,2017,No Data,Central,3,NaN,"2113 Milan Meadows Dr, Leander, TX 78641",NaN,2113 Milan Meadows Dr,Leander,TX,78641,30.530078,-97.908262,https://www.zillow.com/homedetails/2113-Milan-...


In [15]:
df=df[~df.sqft.isna()]
df.reset_index(inplace = True, drop = True)

We check houses that under 1000 sqft and found some houses with only 1 sqft. We corrected it if we have the correct values, otherwise we just drop it.

In [18]:
df[df.sqft<1000]

,beds,baths,sqft,solddate,soldprice,housetype,YearBuilt,Heating,Cooling,parking,lotsize,name,numberOfRooms,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,houseurl
3002,NaN,NaN,1.0,10/6/2017,379990.0,Miscellaneous,2017,Other,None,2,NaN,"929 Mckavitt Dr, Leander, TX 78641",0.0,929 Mckavitt Dr,Leander,TX,78641,30.573400,-97.835156,https://www.zillow.com/homedetails/929-Mckavit...
3175,4.0,2.5,1.0,5/9/2018,195293.0,Single Family,2018,Other,Central,2,NaN,"1141 Peregrine Way, Leander, TX 78641",4.0,1141 Peregrine Way,Leander,TX,78641,30.558426,-97.865868,https://www.zillow.com/homedetails/1141-Peregr...
4997,2.0,2.0,996.0,1/5/2017,154403.0,Single Family,2000,Other,Central,1,3920.0,"1544 Parkfield Cir, Round Rock, TX 78664",2.0,1544 Parkfield Cir,Round Rock,TX,78664,30.490656,-97.659289,https://www.zillow.com/homedetails/1544-Parkfi...


In [19]:
df.loc[df.index==3175,'sqft']=2596
df=df.drop(3002,axis=0)
df.reset_index(inplace = True, drop = True)

## Bathrooms
Before computing the missing values, we first rounded the bathroom values that are not interger nor or halfs.
We then compute the missing bathrooms by choosing its closest sqft.

In [22]:
df.baths.value_counts()

2.00    1971
3.00    1435
2.50     890
4.00     591
3.50     307
5.00      79
4.50      55
5.50       7
7.00       6
6.00       6
2.10       2
3.10       2
4.20       1
1.50       1
2.25       1
2.75       1
Name: baths, dtype: int64

In [23]:
for i in df[df.baths.isin([2.10,3.10,4.20,2.25,2.75])].index:
    df.loc[df.index == i, 'baths']=round(df.loc[df.index == i, 'baths'])

In [24]:
bathagg=df.groupby("baths").agg({"sqft":np.median})
bathagg

,sqft
baths,
1.5,1096.0
2.0,1834.0
2.5,2397.0
3.0,2643.5
3.5,3406.0
4.0,3575.5
4.5,4125.0
5.0,4272.0
5.5,4295.0


In [25]:
for i in df[df.baths.isna()].index:
    minindex = abs(df.loc[df.index == i, 'sqft'].values - bathagg.sqft.values).argmin()
    df.loc[df.index == i, 'baths']=bathagg.index[minindex]


# Bedrooms & number Of Rooms

Before computing missing values, we found both bedrooms and number of Rooms show the same value, which means they are duplicates. Thus, we can safely drop number of rooms and keep bedrooms. Then we compute the missing bedsrooms by choosing its closest sqft.


In [28]:
df.loc[:,["beds","numberOfRooms"]].head()

,beds,numberOfRooms
0,3.0,3.0
1,4.0,4.0
2,3.0,3.0
3,3.0,3.0
4,5.0,5.0


In [29]:
df=df.drop("numberOfRooms",axis=1)

In [30]:
bedsagg=df.groupby("beds").agg({"sqft":np.median})
bedsagg

,sqft
beds,
2.0,1289.0
3.0,1824.0
4.0,2669.0
5.0,3621.0
6.0,3999.5
7.0,4782.0


In [31]:
for i in df[df.beds.isna()].index:
    minindex = abs(df.loc[df.index == i, 'sqft'].values - bedsagg.sqft.values).argmin()
    df.loc[df.index == i, 'beds']=bedsagg.index[minindex]

# Lot size

Before computing missing values, lotsize stats shows that it has extremely high and low values, which need to be checked. We corrected these unlikely values if we are able to find correct values, and dropped those values that are due to **data error**. For example, we found there are many values of <5 are actually in **units of acre,rather than sqft**. Finally, the missing values are computed using **median**.

In [33]:
df.lotsize.describe()

count    4.889000e+03
mean     7.553163e+03
std      7.221603e+04
min      1.100000e-01
25%      5.662000e+03
50%      7.187000e+03
75%      8.407000e+03
max      4.929944e+06
Name: lotsize, dtype: float64

In [35]:
df.iloc[[df.lotsize.idxmax()]]

,beds,baths,sqft,solddate,soldprice,housetype,YearBuilt,Heating,Cooling,parking,lotsize,name,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,houseurl
2200,3.0,3.0,3371.0,1/5/2018,693536.0,Single Family,2013,Other,Central,4,4929944.0,"1809 High Lonesome, Leander, TX 78641",1809 High Lonesome,Leander,TX,78641,30.54375,-97.90039,https://www.zillow.com/homedetails/1809-High-L...


In [36]:
df.loc[df.lotsize==df.lotsize.max(),"lotsize"]=2.02#acre 

In [37]:
#check lotsize under 1000 sqft
df.lotsize[df.lotsize<1000].sort_values(ascending=False)

1459    975.000
2104    960.000
2065    826.000
582     151.000
2029     60.000
1084     60.000
423      50.000
3566     23.000
2429     17.000
1662     11.600
1529      7.000
2439      6.140
1639      6.000
2554      6.000
1268      5.810
1592      5.440
1574      5.260
1789      5.070
3663      5.000
1357      5.000
1215      5.000
1432      5.000
1270      4.960
2132      4.810
1715      4.730
1843      4.610
2004      4.480
1695      4.440
1451      4.140
1299      4.010
         ...   
3408      0.250
3431      0.250
4571      0.250
432       0.250
3615      0.250
4613      0.250
473       0.250
4762      0.250
1279      0.250
1552      0.250
4507      0.250
3260      0.250
3156      0.250
509       0.250
315       0.250
3109      0.250
3125      0.250
3228      0.250
637       0.250
325       0.250
5083      0.250
5080      0.250
3182      0.250
3200      0.250
3983      0.250
1869      0.250
488       0.220
956       0.190
5286      0.165
280       0.110
Name: lotsize, Length: 7

In [38]:
#check until acre>5
df.loc[df.index==1459,"lotsize"]=9736
df.loc[df.index==2104,"lotsize"]=5314
df.loc[df.index==2065,"lotsize"]=5179
df.loc[df.index==582,"lotsize"]=6597
df.loc[df.index==2029,"lotsize"]=7199
df.loc[df.index==1084,"lotsize"]=8799
df.loc[df.index==423,"lotsize"]=6181
df.loc[df.index==3566,"lotsize"]=10267
df.loc[df.index==2429,"lotsize"]=1.73
df.loc[df.index==2554,"lotsize"]=9583 

In [41]:
#corrected the unit from acre to sqft
df["lotsize1"]=df.lotsize
df["lotsize1"][df.lotsize<1000]=df.lotsize[df.lotsize<1000]*43560
df=df.drop("lotsize",axis=1)

In [42]:
df.lotsize1.describe()

count    4.889000e+03
mean     1.120385e+04
std      2.446134e+04
min      1.095000e+03
25%      6.621000e+03
50%      7.840000e+03
75%      9.522000e+03
max      1.106424e+06
Name: lotsize1, dtype: float64

In [43]:
#Corrected data error of 11,064.24 acres
df.lotsize1[df.lotsize1>1000000]
df.loc[df.index==3875,"lotsize1"]=0.25*43560

In [44]:
df.lotsize1.describe()

count      4889.000000
mean      10979.773385
std       18785.865893
min        1095.000000
25%        6621.000000
50%        7840.000000
75%        9522.000000
max      505296.000000
Name: lotsize1, dtype: float64

In [48]:
print ("Lotsize missing data: {}%".format(round((df.lotsize1.isna()).sum()/len(df.lotsize1),2)*100))

Lotsize missing data: 10.0%


In [49]:
#Compute using median
df.lotsize1[df.lotsize1.isna()]=df.lotsize1.median()

In [51]:
(df.isna()).sum()

beds               0
baths              0
sqft               0
solddate           0
soldprice          0
housetype          0
YearBuilt          0
Heating            0
Cooling            0
parking            0
name               0
streetAddress      0
addressLocality    0
addressRegion      0
postalCode         0
latitude           0
longitude          0
houseurl           0
lotsize1           0
dtype: int64

# Sold Price
Although there is no missing value found in soldprice, we can from below that we have both extremely high and low values. Soldprices under 150k and above 1000k due to data error were dropped .

In [52]:
df.soldprice.describe()

count    5.440000e+03
mean     3.104395e+05
std      1.306900e+05
min      1.000000e+00
25%      2.328648e+05
50%      2.766705e+05
75%      3.600000e+05
max      2.415000e+06
Name: soldprice, dtype: float64

In [59]:
df=df.drop(df[df.soldprice<150000].index)#under $150k due to data error
df=df.drop(1718) #above $1000k due to data error

In [61]:
df.shape

(5308, 19)

# Save data for next part
Finally, we dropped url that won't be need later and double checked we don't have missing values. We saved the data for the next part.

In [64]:
df_clean=df.drop(["houseurl"],axis=1)

In [65]:
(df.isna()).sum()

beds               0
baths              0
sqft               0
solddate           0
soldprice          0
housetype          0
YearBuilt          0
Heating            0
Cooling            0
parking            0
name               0
streetAddress      0
addressLocality    0
addressRegion      0
postalCode         0
latitude           0
longitude          0
houseurl           0
lotsize1           0
dtype: int64

In [66]:
df_clean.to_csv("df_clean.csv",index=False)